<a href="https://colab.research.google.com/github/ozguozkan/DI725_Assign2/blob/main/DI725_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

connecting to the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

importing necessary libs

In [ ]:
import os
import json
from __future__ import annotations
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import random_split
import wandb
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

wandb login

In [ ]:
!pip install -q wandb

In [ ]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ozgukan to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

getting the detr_resnet50 model

In [ ]:
!pip install -q transformers

from transformers import DetrForObjectDetection, DetrImageProcessor

my paths in colab

In [ ]:
path = '/content/drive/MyDrive/auair2019'
img_path = os.path.join(path, 'images')
annot_path = os.path.join(path, 'annotations.json')

reading the annotations

In [ ]:
with open(annot_path, 'r') as f:
  annotations = json.load(f)

print(json.dumps(annotations['annotations'][0], indent=2)) #seeing the first images info


{
  "image_name": "frame_20190829091111_x_0001973.jpg",
  "image_width:": 1920.0,
  "image_height": 1080.0,
  "platform": "Parrot Bebop 2",
  "time": {
    "year": 2019,
    "month": 8,
    "day": 29,
    "hour": 9,
    "min": 11,
    "sec": 11,
    "ms": 394400.0
  },
  "longtitude": 10.18798203255313,
  "latitude": 56.20630134795274,
  "altitude": 19921.6,
  "linear_x": 0.03130074199289083,
  "linear_y": 0.028357808757573367,
  "linear_z": 0.0744575835764408,
  "angle_phi": -0.06713105738162994,
  "angle_theta": 0.06894744634628296,
  "angle_psi": 1.1161083340644837,
  "bbox": [
    {
      "top": 163,
      "left": 1098,
      "height": 185,
      "width": 420,
      "class": 1
    },
    {
      "top": 421,
      "left": 1128,
      "height": 176,
      "width": 393,
      "class": 1
    },
    {
      "top": 927,
      "left": 1703,
      "height": 153,
      "width": 183,
      "class": 0
    }
  ]
}


In [ ]:
print("Toplam annotation sayısı:", len(dataset.annotations))

Toplam annotation sayısı: 32823


i need to fix image_width: problem

i need to change the annotaiton format into  (x min, y min, x max, y max) and i ned the match each annotation with its image

In [ ]:
image_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

class_mapping = {
    1: 0,  # human
    2: 1,  # car
    3: 2,  # truck
    4: 3,  # van
    5: 4,  # motorbike
    6: 5,  # bicycle
    7: 6,  # bus
    8: 7   # trailer
}

class auairdata(Dataset):
  def __init__(self, annot_path, img_dir, image_processor):
    with open(annot_path, 'r') as f:
      all_data = json.load(f)
      self.annotations = all_data["annotations"]

    self.img_dir = img_dir
    # self.transform = transform
    self.image_processor = image_processor

    self.converted_annotations = []

    self.image_name_to_id = {
            name: idx for idx, name in enumerate(
                sorted(set([ann["image_name"] for ann in self.annotations]))
            )
        }



    #to make my data compatible with the hw i change the variables
    #pytorch expects the data as x_min, y_max etc.
    for annot in self.annotations:
      image_id = self.image_name_to_id[annot["image_name"]]
      for bbox in annot["bbox"]:
        x = bbox["left"]
        y = bbox["top"]
        w = bbox["width"]
        h = bbox["height"]
        original_cls = bbox["class"]

        if original_cls in class_mapping:
          cls = class_mapping[original_cls]
        else:
          continue

        self.converted_annotations.append({
            "image_id": image_id,
            "bbox": [x, y, w, h],
            "category_id": cls,
            "area": w * h,
            "iscrowd": 0
        })

      self.image_names = [annot["image_name"] for annot in self.annotations]

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, idx): #the annotation and the image has been matched
    annot = self.annotations[idx]
    img_path = os.path.join(self.img_dir, annot["image_name"])
    image = Image.open(img_path).convert("RGB")

    image_id = self.image_name_to_id[annot["image_name"]]
    coco_labels = [ann for ann in self.converted_annotations if ann ["image_id"] == image_id]


    mytargets = { #because torch.vision expects the boxes and labels in this format
        # 'image_id': torch.tensor([idx]),
        'image_id': image_id,
        'annotations': coco_labels
    }

    encode = image_processor(image, annotations=mytargets, return_tensors="pt")

    return {
        "pixel_values": encode["pixel_values"].squeeze(0),
        "labels": encode["labels"][0]
    }

def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    labels = [item["labels"] for item in batch]
    return {
        "pixel_values": torch.stack(pixel_values),
        "labels": labels
    }
dataset = auairdata(
    annot_path=annot_path,
    img_dir=img_path,
    image_processor=image_processor
)

# DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)

#saving as coco
save_path = os.path.join(path, 'converted_annotations.json')
with open(save_path, 'w') as f:
    json.dump({"annotations": dataset.converted_annotations}, f, indent=2)
print(f"json saved: {save_path}")

#matching process
with open(annot_path, 'r') as f:
    auair_data = json.load(f)

image_id_to_name = {}
for annot in auair_data["annotations"]:
    image_id = annot.get("image_id")
    image_name = annot.get("image_name")
    if image_id is not None and image_name is not None:
        image_id_to_name[image_id] = image_name

with open(save_path, 'r') as f:
    converted = json.load(f)

for ann in converted["annotations"]:
    img_id = ann["image_id"]
    ann["image_name"] = image_id_to_name.get(img_id, f"frame_{img_id:07d}.jpg")

new_save_path = os.path.join(path, 'converted_annotations_with_names.json')
with open(new_save_path, 'w') as f:
    json.dump(converted, f, indent=2)
print(f"new json saved: {new_save_path}")


json saved: /content/drive/MyDrive/auair2019/converted_annotations.json
new json saved: /content/drive/MyDrive/auair2019/converted_annotations_with_names.json


In [ ]:
class CocoStyleDataset(Dataset):
    def __init__(self, annotation_path, img_dir, image_processor, transform=None):
        with open(annotation_path, 'r') as f:
            all_data = json.load(f)
            self.annotations = all_data["annotations"]

        self.img_dir = img_dir
        self.image_processor = image_processor
        self.transform = transform

        # if there is image name group them
        self.image_to_annots = {}
        for ann in self.annotations:
            img_name = ann["image_name"]
            self.image_to_annots.setdefault(img_name, []).append(ann)

        self.image_names = sorted(self.image_to_annots.keys())

        self.image_name_to_id = {
            name: idx for idx, name in enumerate(self.image_names)
        }

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        img_path = os.path.join(self.img_dir, image_name)
        image = Image.open(img_path).convert("RGB")

        annots = self.image_to_annots[image_name]
        image_id = self.image_name_to_id[image_name]

        mytargets = {
            "image_id": image_id,
            "annotations": annots
        }

        encode = image_processor(image, annotations=mytargets, return_tensors="pt")

        return {
            "pixel_values": encode["pixel_values"].squeeze(0),
            "labels": encode["labels"][0]
        }


In [ ]:
dataset = CocoStyleDataset(
    annotation_path='/content/drive/MyDrive/auair2019/converted_annotations_with_names.json',
    img_dir='/content/drive/MyDrive/auair2019/images',
    image_processor=image_processor
)

In [ ]:
import json

with open('/content/drive/MyDrive/auair2019/converted_annotations_with_names.json', 'r') as f:
    data = json.load(f)

category_ids = [ann["category_id"] for ann in data["annotations"]]
print("Eşsiz sınıf ID'leri:", sorted(set(category_ids)))
print("Toplam sınıf sayısı:", len(set(category_ids)))


Eşsiz sınıf ID'leri: [0, 1, 2, 3, 4, 5, 6, 7]
Toplam sınıf sayısı: 8


In [ ]:
import random

sample_ann = random.choice(data["annotations"])
print(json.dumps(sample_ann, indent=2))

{
  "image_id": 19042,
  "bbox": [
    1207,
    237,
    206,
    144
  ],
  "category_id": 2,
  "area": 29664,
  "iscrowd": 0,
  "image_name": "frame_20190905112522_x_0001297.jpg"
}


checking how many classes are there in the auair datasaet

In [ ]:
with open('/content/drive/MyDrive/auair2019/annotations.json', 'r') as f:
    data = json.load(f)

annotations = data["annotations"]  # my annot list is here

class_ids = set()

for ann in annotations:
    for bbox in ann["bbox"]:
        class_ids.add(bbox["class"])

print(f"total class num: {len(class_ids)}")
print(f"classes: {sorted(class_ids)}")


total class num: 8
classes: [0, 1, 2, 3, 4, 5, 6, 7]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

#as we seen before in auair dataset there are 8 classes and the backgorund so 9 classes
#i need to adjust the class number for the detr model
#the detr also wants tje background thats  why i add it

num_classes = 9

model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels=num_classes,  # class num
    ignore_mismatched_sizes=True
)

model.to(device)
model.train()

id2label = {
    0: "human",
    1: "car",
    2: "truck",
    3: "van",
    4: "motorbike",
    5: "bicycle",
    6: "bus",
    7: "trailer",
    8: "background"
}
label2id = {v: k for k, v in id2label.items()}

model.config.id2label = id2label
model.config.label2id = label2id


device: cuda


config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

splitting the dataset

In [ ]:
#%70 train, %15 val, %15 test
ful_len = len(dataset)
train_len = int(0.7*ful_len)
val_len = int(0.15*ful_len)
test_len = ful_len - train_len - val_len

print(f"train: {train_len}, val: {val_len}, test:{test_len}")

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_len, val_len, test_len],
    generator=torch.Generator().manual_seed(42)
)

#i add num workers an pin memory bc 4 epochs lasted 11.30 hours
my_loader = {
    "batch_size": 16,
    "num_workers": 4,
    "pin_memory": True,
    "collate_fn": collate_fn
}

# 🚀 DataLoaders
train_loader = DataLoader(train_dataset, shuffle=True, **my_loader)
val_loader   = DataLoader(val_dataset,   shuffle=False, **my_loader)
test_loader  = DataLoader(test_dataset,  shuffle=False, **my_loader)

train: 22493, val: 4819, test:4821


In [ ]:
wandb.init(project = "DI725-Assignment2", name = "detr-finetune", config = {
    "epochs": 15,
    "batch_size": 16,
    "lr": 5e-5
})


model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels = 9,
    ignore_mismatched_sizes = True).to(device)

id2label = {
    0: "human", 1: "car", 2: "truck", 3: "van",
    4: "motorbike", 5: "bicycle", 6: "bus", 7: "trailer", 8: "background"
}
label2id = {v: k for k, v in id2label.items()}
model.config.id2label = id2label
model.config.label2id = label2id

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

train_losses = []
val_losses = []
best_val_loss = float("inf")

def evaluate(model, dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in dataloader:
            pixel_values = batch["pixel_values"].to(device)
            labels = [
                {k: v.to(device).float() if k == "boxes" else v.to(device)
                 for k, v in l.items()} for l in batch["labels"]
            ]
            outputs = model(pixel_values=pixel_values, labels=labels)
            total_loss += outputs.loss.item()
    return total_loss / len(dataloader)

#train
for epoch in range(wandb.config["epochs"]):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{wandb.config['epochs']}", leave=False)

    for batch in train_bar:
        pixel_values = batch["pixel_values"].to(device)
        labels = [
            {k: v.to(device).float() if k == "boxes" else v.to(device)
             for k, v in l.items()} for l in batch["labels"]
        ]

        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_bar.set_postfix({"Loss": loss.item()})

    train_loss = running_loss / len(train_loader)
    val_loss = evaluate(model, val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss
    })

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

#saving the best model
    # En iyi modeli kaydet
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_detr_model.pth")
        print(f"[Epoch {epoch+1}] best model saved (val_loss: {val_loss:.4f})")
        wandb.save("best_detr_model.pth")

#evaluating test performance
test_loss = evaluate(model, test_loader)
wandb.log({"test_loss": test_loss})
print(f"test loss: {test_loss:.4f}")

#plot
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
plt.plot(range(1, len(val_losses)+1), val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("train vs val loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

wandb.log({
    "train_val_loss_curve": wandb.plot.line_series(
        xs=list(range(1, len(train_losses)+1)),
        ys=[train_losses, val_losses],
        keys=["Train", "Val"],
        title="Loss Over Epochs",
        xname="Epoch"
    )
})

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 420, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 420, in <listcomp>
    return [self.dataset[self.indices[idx]] for idx in indices]
            ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  Fil

In [ ]:
import os

missing = []
for ann in dataset.annotations:
    img_name = ann["image_name"]
    img_path = os.path.join(img_path, img_name)
    if not os.path.exists(img_path):
        missing.append(img_name)

print(f"Eksik görsellerin sayısı: {len(missing)}")
print("İlk 5 eksik:", missing[:5])


Eksik görsellerin sayısı: 126873
İlk 5 eksik: ['frame_0001292.jpg', 'frame_0001292.jpg', 'frame_0000178.jpg', 'frame_0001872.jpg', 'frame_0001872.jpg']


In [ ]:
import os

img_dir = "/content/drive/MyDrive/auair2019/images"
image_files = [f for f in os.listdir(img_dir) if f.endswith(".jpg")]
print(f"📸 Toplam görüntü sayısı: {len(image_files)}")


📸 Toplam görüntü sayısı: 32823


In [ ]:
import json

json_path = "/content/drive/MyDrive/auair2019/converted_annotations_with_names.json"

with open(json_path, "r") as f:
    data = json.load(f)

image_names_in_json = set(ann["image_name"] for ann in data["annotations"])
print(f"📄 JSON'daki benzersiz image_name sayısı: {len(image_names_in_json)}")


📄 JSON'daki benzersiz image_name sayısı: 32133
